# Predictiv Maintenance

In [179]:
# Import Libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mlp
import matplotlib.pyplot as plt
import scipy
import math
from datetime import time, timedelta
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Matplotlib Default Parameter

from matplotlib import rcParams
plt.style.use('fivethirtyeight')
plt.rcParams['axes.prop_cycle'] = plt.rcParamsDefault['axes.prop_cycle']


# display all columns
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [62]:
# read data
DBR_df = pd.read_excel("D:\document\data_science\DLSL Study case\Final_Project\All Data DBR 2022 - Unit Random.xlsx")

In [164]:
# display tabel
DBR_df.head()

,SITE,MONTH,LOKASI UNIT,SECTION,NO,EGI,CODE NUMBER UNIT,HM,PROBLEM DESCRIPTION,START B/D (DATE),START B/D (TIME),WO NUMBER,WO / BD TYPE,MAINTENANCE TYPE,STATUS UNIT,ACTION PROBLEM,ACTIVITY CODE,DESCRIPTION ACTIVITY,START ACTIVITY (DATE),START ACTIVITY (TIME),FINISH ACTIVITY (DATE),FINISH ACTIVITY (TIME),RFU (DATE),RFU (TIME),AGING B/D (DAY),AGING B/D (HOUR),AGING ACTIVITY (DAY),AGING ACTIVITY (HOUR),SUB COMP CODE,COMP CODE,DESCRIPTION SUB COMP (PA),DESRICTION COMP (PA),DESCRIPTION SUB COMP (MTBF),DESRICTION COMP (MTBF),PIC MEKANIK / GROUP LEADER,RO / PR NUMBER,RO / PR (DATE),RO / PR (STATUS),PO NUMBER,PO (DATE),PO (STATUS),REMARK
0,SITE D,May,NaN,WHEEL,88,EGI 52,CN 29,15593,AIR PRESSURE DROP,2022-05-06 00:00:00,8.000000,300100060749,UNSCHEDULE,RP,RFU,WAITING SARANA,B12,WAITING TRANSPORTATION,2022-05-06,0.340278,2022-05-06,0.379861,2022-05-06 00:00:00,0.379861,0.0,NaN,0.0,0.950000,7500.0,7000.0,Air Piping and Hose,PNEUMATIC SYSTEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SITE D,May,NaN,WHEEL,NaN,EGI 52,CN 29,15593,AIR PRESSURE DROP,2022-05-06 00:00:00,8.000000,300100060749,UNSCHEDULE,RP,RFU,REPAIR AIR HOSE,B9,ON PROCESS,2022-05-06,0.340278,2022-05-06,0.390278,2022-05-06 00:00:00,0.390278,0.0,NaN,0.0,1.200000,7500.0,7000.0,Air Piping and Hose,PNEUMATIC SYSTEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SITE A,January,NaN,NaN,NaN,EGI 47,CN 473,4947,TYRE NO. 7 LEAK,2022-01-04 00:00:00,0.694444,TYR04012022-11,SCHEDULE,RIT,CONTINUE,REPLACE TYRE NO. 7,B9,ON PROCESS,2022-01-04,0.694444,2022-01-04,0.750000,NaN,NaN,NaN,NaN,0.0,1.333333,6321.0,6000.0,Tyre,UNDERCARRIAGE & TIRE,NaN,NaN,"TAJU, ABDULLAH",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SITE B,January,NaN,WHEEL BIG,134,EGI 12,CN 348,17722,TYRE NO. 5 LEAK,2022-01-03 00:00:00,0.642361,TYR03012021-11,UNSCHEDULE,RPT,CONTINUE,REPLACE TYRE NO. 5,B9,ON PROCESS,2022-01-03,0.642361,2022-01-03,0.750000,NaN,NaN,NaN,NaN,0.0,2.583333,6321.0,6000.0,Tyre,UNDERCARRIAGE & TIRE,Tyre,UNDERCARRIAGE & TIRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SITE A,December,NaN,WHEEL BIG,NaN,EGI 12,CN 343,21360,OIL ENGINE MIX COOLANT ( CYL NO 11 LEAK ),2022-12-08 00:00:00,0.275000,300100080306,UNSCHEDULE,RP,CONTINUE,"REMOVE ENGINE, CLEANING FRONT ENGINE",B9,ON PROCESS,2022-12-18,0.750000,2022-12-19,0.270833,NaN,NaN,NaN,NaN,1.0,12.500000,1920.0,1000.0,Exhaust Muffler,ENGINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUJAN


In [199]:
# filter only EGI 12 machine
EGI12_df = DBR_df[DBR_df["EGI"] == "EGI 12"]
EGI12_df.head()

,SITE,MONTH,LOKASI UNIT,SECTION,NO,EGI,CODE NUMBER UNIT,HM,PROBLEM DESCRIPTION,START B/D (DATE),START B/D (TIME),WO NUMBER,WO / BD TYPE,MAINTENANCE TYPE,STATUS UNIT,ACTION PROBLEM,ACTIVITY CODE,DESCRIPTION ACTIVITY,START ACTIVITY (DATE),START ACTIVITY (TIME),FINISH ACTIVITY (DATE),FINISH ACTIVITY (TIME),RFU (DATE),RFU (TIME),AGING B/D (DAY),AGING B/D (HOUR),AGING ACTIVITY (DAY),AGING ACTIVITY (HOUR),SUB COMP CODE,COMP CODE,DESCRIPTION SUB COMP (PA),DESRICTION COMP (PA),DESCRIPTION SUB COMP (MTBF),DESRICTION COMP (MTBF),PIC MEKANIK / GROUP LEADER,RO / PR NUMBER,RO / PR (DATE),RO / PR (STATUS),PO NUMBER,PO (DATE),PO (STATUS),REMARK
3,SITE B,January,NaN,WHEEL BIG,134,EGI 12,CN 348,17722,TYRE NO. 5 LEAK,2022-01-03 00:00:00,0.642361,TYR03012021-11,UNSCHEDULE,RPT,CONTINUE,REPLACE TYRE NO. 5,B9,ON PROCESS,2022-01-03,0.642361,2022-01-03,0.750000,NaN,NaN,NaN,NaN,0.0,2.583333,6321.0,6000.0,Tyre,UNDERCARRIAGE & TIRE,Tyre,UNDERCARRIAGE & TIRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SITE A,December,NaN,WHEEL BIG,NaN,EGI 12,CN 343,21360,OIL ENGINE MIX COOLANT ( CYL NO 11 LEAK ),2022-12-08 00:00:00,0.275000,300100080306,UNSCHEDULE,RP,CONTINUE,"REMOVE ENGINE, CLEANING FRONT ENGINE",B9,ON PROCESS,2022-12-18,0.750000,2022-12-19,0.270833,NaN,NaN,NaN,NaN,1.0,12.500000,1920.0,1000.0,Exhaust Muffler,ENGINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUJAN
9,SITE B,October,NaN,WHEEL BIG,NaN,EGI 12,CN 346,20358,SCH. CHECK ENGINE,2022-09-20 00:00:00,0.059028,300200031322,SCHEDULE,BL,CONTINUE,"CHECK PIPING EXHAUST, CHECK CYL HEAD BY TRAKINDO",B9,ON PROCESS,2022-10-01,0.750000,2022-10-02,0.270833,NaN,NaN,NaN,NaN,1.0,12.500000,1810.0,1000.0,Cylinder Block & Liner,ENGINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,SITE A,January,NaN,WHEEL BIG,291,EGI 12,CN 342,14258,"PS I, SCH. RE-SEAL CYLINDER STEERING, SCH. REP...",2022-01-20 00:00:00,0.304167,300500017390,SCHEDULE,PS,CONTINUE,NaN,B9,ON PROCESS,2022-01-20,0.304167,2022-01-20,0.750000,NaN,NaN,NaN,NaN,0.0,10.700000,9600.0,9600.0,PERIODIC SERVICE,PERIODIC SERVICE,PERIODIC SERVICE,PERIODIC SERVICE,"BAGUS, ALIF",NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,SITE A,January,NaN,NaN,NaN,EGI 12,CN 342,14258,"PS I, SCH. RE-SEAL CYLINDER STEERING, \nSCH. R...",2022-01-20 00:00:00,0.304167,300500017390,SCHEDULE,PS,CONTINUE,NaN,B9,ON PROCESS,2022-01-20,0.750000,2022-01-21,0.270833,NaN,NaN,NaN,NaN,1.0,12.500000,9600.0,9600.0,PERIODIC SERVICE,PERIODIC SERVICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
# list coloums name in table 

EGI12_df.columns

Index(['SITE', 'MONTH', 'LOKASI UNIT', 'SECTION', 'NO', 'EGI',
       'CODE NUMBER UNIT', 'HM', 'PROBLEM DESCRIPTION', 'START B/D (DATE)',
       'START B/D (TIME)', 'WO NUMBER', 'WO / BD TYPE', 'MAINTENANCE TYPE',
       'STATUS UNIT', 'ACTION PROBLEM', 'ACTIVITY CODE',
       'DESCRIPTION ACTIVITY', 'START ACTIVITY (DATE)',
       'START ACTIVITY (TIME)', 'FINISH ACTIVITY (DATE)',
       'FINISH ACTIVITY (TIME)', 'RFU (DATE)', 'RFU (TIME)', 'AGING B/D (DAY)',
       'AGING B/D (HOUR)', 'AGING ACTIVITY (DAY)', 'AGING ACTIVITY (HOUR)',
       'SUB COMP CODE', 'COMP CODE', 'DESCRIPTION SUB COMP (PA)',
       'DESRICTION COMP (PA)', 'DESCRIPTION SUB COMP (MTBF)',
       'DESRICTION COMP (MTBF)', 'PIC MEKANIK / GROUP LEADER',
       'RO / PR NUMBER', 'RO / PR (DATE)', 'RO / PR (STATUS)', 'PO NUMBER',
       'PO (DATE)', 'PO (STATUS)', 'REMARK'],
      dtype='object')

In [212]:
# select only imnportan coloum (Research)

important_coloum = ['EGI','CODE NUMBER UNIT','HM','PROBLEM DESCRIPTION','START B/D (DATE)','START B/D (TIME)',
                    'WO / BD TYPE','ACTION PROBLEM','ACTIVITY CODE','DESCRIPTION ACTIVITY','START ACTIVITY (DATE)',
                    'START ACTIVITY (TIME)', 'FINISH ACTIVITY (DATE)','FINISH ACTIVITY (TIME)',
                    'RFU (DATE)','AGING B/D (DAY)','AGING B/D (HOUR)','COMP CODE','DESRICTION COMP (PA)'
                    ]
EGI12_df = EGI12_df[important_coloum]

In [226]:
# Create Data Info

list_item = []
for col in EGI12_df.columns:
    list_item.append([col, EGI12_df[col].dtype, EGI12_df[col].isna().sum(), 100*EGI12_df[col].isna().sum()/len(EGI12_df[col]), EGI12_df[col].nunique(), EGI12_df[col].unique()[:4]])
desc_df = pd.DataFrame(data=list_item, columns='feature data_type null_num null_pct unique_num unique_sample'.split())
desc_df

,feature,data_type,null_num,null_pct,unique_num,unique_sample
0,EGI,object,0,0.000000,1,[EGI 12]
1,CODE NUMBER UNIT,object,0,0.000000,33,"[CN 348, CN 343, CN 346, CN 342]"
2,HM,object,21,0.521999,1659,"[17722, 21360, 20358, 14258]"
3,PROBLEM DESCRIPTION,object,0,0.000000,1107,"[TYRE NO. 5 LEAK, OIL ENGINE MIX COOLANT ( CYL..."
4,START B/D (DATE),object,0,0.000000,361,"[2022-01-03 00:00:00, 2022-12-08 00:00:00, 202..."
5,START B/D (TIME),object,0,0.000000,509,"[06:25:00, 02:44:00, 00:35:00, 03:02:00]"
6,WO / BD TYPE,object,0,0.000000,2,"[UNSCHEDULE, SCHEDULE]"
7,ACTION PROBLEM,object,193,4.797415,1861,"[REPLACE TYRE NO. 5, REMOVE ENGINE, CLEANING F..."
8,ACTIVITY CODE,object,1,0.024857,18,"[B9, B5, B14, B11]"
9,DESCRIPTION ACTIVITY,object,1,0.024857,20,"[ON PROCESS, WAITING PART, WAITING PROGRESS BY..."


- there is missing value in the feature ( `HM`, `ACTION PROBLEM`, `ACTIVITY CODE`, `DESCRIPTION ACTIVITY`, `FINISH ACTIVITY (DATE)`,<br> `FINISH ACTIVITY (TIME)`, `RFU (DATE)`,`A GING B/D (DAY)` and `AGING B/D (HOUR)`).

- need to convert object feature to time (`START B/D (DATE)`, `START B/D (TIME)`, `START ACTIVITY (TIME)` and `FINISH ACTIVITY (TIME)`).

- need to convert `WO / BD TYPE` to boolean 

In [203]:
def time_manipulation(x):
    """functi yang mentrasformasi waktu dengan merujuk waktu kerja dari jam 08.00 hingga 18.00"""
    if x >= 8 and x <= 1:
        decimal_hours = x*10
        hours = int(decimal_hours)
        minutes = int((decimal_hours - hours) * 60)
        t = time(hours, minutes)
        return t
    elif x >= 0.12 and x <= 0.24:
        decimal_hours = x*100
        hours = int(decimal_hours)
        minutes = int((decimal_hours - hours) * 60)
        t = time(hours, minutes)
        return t
    else:
        decimal_hours = x*10
        hours = int(decimal_hours)
        minutes = int((decimal_hours - hours) * 60)
        t = time(hours, minutes)
        return t

In [204]:
EGI12_df.columns

Index(['EGI', 'CODE NUMBER UNIT', 'HM', 'PROBLEM DESCRIPTION',
       'START B/D (DATE)', 'START B/D (TIME)', 'WO / BD TYPE',
       'ACTION PROBLEM', 'ACTIVITY CODE', 'DESCRIPTION ACTIVITY',
       'START ACTIVITY (DATE)', 'START ACTIVITY (TIME)',
       'FINISH ACTIVITY (DATE)', 'FINISH ACTIVITY (TIME)', 'RFU (DATE)',
       'AGING B/D (DAY)', 'AGING B/D (HOUR)', 'COMP CODE',
       'DESRICTION COMP (PA)'],
      dtype='object')

In [224]:
EGI12_df_1 = EGI12_df[EGI12_df['FINISH ACTIVITY (TIME)'].isnull()].dropna()

In [205]:
for i in ['START B/D (TIME)','START ACTIVITY (TIME)','FINISH ACTIVITY (TIME)']:
    EGI12_df[i] = EGI12_df[i].apply(time_manipulation)

ValueError: cannot convert float NaN to integer

In [186]:
EGI12_df['START B/D (TIME)'] = EGI12_df['START B/D (TIME)'].apply(time_manipulation)
EGI12_df['START ACTIVITY (TIME)'] = EGI12_df['START ACTIVITY (TIME)'].apply(time_manipulation)
EGI12_df['FINISH ACTIVITY (TIME)'] = EGI12_df['FINISH ACTIVITY (TIME))'].apply(time_manipulation)

In [187]:
EGI12_df.head()

,EGI,CODE NUMBER UNIT,HM,PROBLEM DESCRIPTION,START B/D (DATE),START B/D (TIME),WO / BD TYPE,ACTION PROBLEM,ACTIVITY CODE,DESCRIPTION ACTIVITY,START ACTIVITY (DATE),START ACTIVITY (TIME),FINISH ACTIVITY (DATE),FINISH ACTIVITY (TIME),RFU (DATE),AGING B/D (DAY),AGING B/D (HOUR),COMP CODE,DESRICTION COMP (PA)
3,EGI 12,CN 348,17722,TYRE NO. 5 LEAK,2022-01-03 00:00:00,06:25:00,UNSCHEDULE,REPLACE TYRE NO. 5,B9,ON PROCESS,2022-01-03,0.642361,2022-01-03,0.750000,NaN,NaN,NaN,6000.0,UNDERCARRIAGE & TIRE
4,EGI 12,CN 343,21360,OIL ENGINE MIX COOLANT ( CYL NO 11 LEAK ),2022-12-08 00:00:00,02:44:00,UNSCHEDULE,"REMOVE ENGINE, CLEANING FRONT ENGINE",B9,ON PROCESS,2022-12-18,0.750000,2022-12-19,0.270833,NaN,NaN,NaN,1000.0,ENGINE
9,EGI 12,CN 346,20358,SCH. CHECK ENGINE,2022-09-20 00:00:00,00:35:00,SCHEDULE,"CHECK PIPING EXHAUST, CHECK CYL HEAD BY TRAKINDO",B9,ON PROCESS,2022-10-01,0.750000,2022-10-02,0.270833,NaN,NaN,NaN,1000.0,ENGINE
32,EGI 12,CN 342,14258,"PS I, SCH. RE-SEAL CYLINDER STEERING, SCH. REP...",2022-01-20 00:00:00,03:02:00,SCHEDULE,NaN,B9,ON PROCESS,2022-01-20,0.304167,2022-01-20,0.750000,NaN,NaN,NaN,9600.0,PERIODIC SERVICE
33,EGI 12,CN 342,14258,"PS I, SCH. RE-SEAL CYLINDER STEERING, \nSCH. R...",2022-01-20 00:00:00,03:02:00,SCHEDULE,NaN,B9,ON PROCESS,2022-01-20,0.750000,2022-01-21,0.270833,NaN,NaN,NaN,9600.0,PERIODIC SERVICE


In [190]:
EGI12_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4023 entries, 3 to 73663
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EGI                     4023 non-null   object        
 1   CODE NUMBER UNIT        4023 non-null   object        
 2   HM                      4002 non-null   object        
 3   PROBLEM DESCRIPTION     4023 non-null   object        
 4   START B/D (DATE)        4023 non-null   object        
 5   START B/D (TIME)        4023 non-null   object        
 6   WO / BD TYPE            4023 non-null   object        
 7   ACTION PROBLEM          3830 non-null   object        
 8   ACTIVITY CODE           4022 non-null   object        
 9   DESCRIPTION ACTIVITY    4022 non-null   object        
 10  START ACTIVITY (DATE)   4023 non-null   datetime64[ns]
 11  START ACTIVITY (TIME)   4023 non-null   float64       
 12  FINISH ACTIVITY (DATE)  4020 non-null   datetim

In [133]:
import datetime

# float value representing time
time_float = 13.05

# convert float to time
time_object = datetime.datetime.strptime(str(time_float), "%H.%M")

# format the time as a string in hour:minute format
time_string = time_object.strftime("%H:%M")

print(time_string) # Output: "13:05"


13:05


In [138]:
type(time_float)

float

In [144]:
EGI12_df

,EGI,CODE NUMBER UNIT,HM,PROBLEM DESCRIPTION,START B/D (DATE),START B/D (TIME),WO / BD TYPE,ACTION PROBLEM,ACTIVITY CODE,DESCRIPTION ACTIVITY,RFU (DATE),AGING B/D (DAY),AGING B/D (HOUR),COMP CODE,DESRICTION COMP (PA)
3,EGI 12,CN 348,17722,TYRE NO. 5 LEAK,2022-01-03 00:00:00,6.42,UNSCHEDULE,REPLACE TYRE NO. 5,B9,ON PROCESS,NaN,NaN,NaN,6000.0,UNDERCARRIAGE & TIRE
4,EGI 12,CN 343,21360,OIL ENGINE MIX COOLANT ( CYL NO 11 LEAK ),2022-12-08 00:00:00,2.75,UNSCHEDULE,"REMOVE ENGINE, CLEANING FRONT ENGINE",B9,ON PROCESS,NaN,NaN,NaN,1000.0,ENGINE
9,EGI 12,CN 346,20358,SCH. CHECK ENGINE,2022-09-20 00:00:00,0.59,SCHEDULE,"CHECK PIPING EXHAUST, CHECK CYL HEAD BY TRAKINDO",B9,ON PROCESS,NaN,NaN,NaN,1000.0,ENGINE
32,EGI 12,CN 342,14258,"PS I, SCH. RE-SEAL CYLINDER STEERING, SCH. REP...",2022-01-20 00:00:00,3.04,SCHEDULE,NaN,B9,ON PROCESS,NaN,NaN,NaN,9600.0,PERIODIC SERVICE
33,EGI 12,CN 342,14258,"PS I, SCH. RE-SEAL CYLINDER STEERING, \nSCH. R...",2022-01-20 00:00:00,3.04,SCHEDULE,NaN,B9,ON PROCESS,NaN,NaN,NaN,9600.0,PERIODIC SERVICE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73615,EGI 12,CN 343,20690,"SUSPENSI DEPAN KIRI KURANG GREASE, SUSPENSI BE...",2022-10-24 00:00:00,0.00,SCHEDULE,ADJUST SUSPENSI,B9,ON PROCESS,2022-10-24 00:00:00,0.0,0.000000,6000.0,UNDERCARRIAGE & TIRE
73619,EGI 12,CN 345,21504,PS II,2022-11-01 00:00:00,3.75,SCHEDULE,REPLACE FUEL PRO,B9,ON PROCESS,2022-11-01 00:00:00,0.0,0.000000,1000.0,ENGINE
73627,EGI 12,CN 188,2562,Error Panel Brake,2022-08-08 00:00:00,7.92,SCHEDULE,Repair Wiring Sensor Brake,B9,ON PROCESS,2022-08-08 00:00:00,0.0,0.000000,8000.0,ELECTRICAL SYSTEM
73645,EGI 12,CN 456,NaN,Connector Hose Steering Leaj,2022-10-10 00:00:00,7.50,SCHEDULE,Waiting Mekanik Trakindo,B8,WAITING MEKANIK,2022-10-10 00:00:00,0.0,0.000000,5000.0,BRAKING & STEERING SYSTEM


In [140]:
from datetime import datetime

# float value representing time
time_float = 13.05

# convert float to string
time_string = str(time_float)

# format string
format_string = "%H.%M"

# parse string and create datetime object
time_object = datetime.strptime(time_string, format_string)

# format the time as a string in hour:minute format
time_string = time_object.strftime("%H:%M")

print(time_string) # Output: "13:05"



13:05
